---
title: Time Steppers with Diffrax
suptitle: Diffraxing
short_title: Time Step
date: 2023-02-21
authors:
  - name: J. Emmanuel Johnson
    affiliations:
      - MEOM Lab
    email: jemanjohnson34@gmail.com
license: CC-BY-4.0
keywords: PDE, ODE, TimeStepper, Diffrax
---

+++ {"part": "abstract"}

We introduce....

+++

**Citation**

follow the FORCE11 recommendations [hi](doi:10.4230/DAGMAN.1.1.41). Specifically: